In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import re

In [2]:
train = pd.read_csv(r"C:\Users\raven\Downloads\sf-crime\train.csv\train.csv")
test  = pd.read_csv(r"C:\Users\raven\Downloads\sf-crime\test.csv\test.csv")

In [3]:
train1 = train.drop(['Descript', 'Resolution'], axis = 1)
train1.head()

,Dates,Category,DayOfWeek,PdDistrict,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,Wednesday,NORTHERN,VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,Wednesday,NORTHERN,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,Wednesday,PARK,100 Block of BRODERICK ST,-122.438738,37.771541


In [4]:
train1 = train1.query('Y != 90')

In [5]:
Dt = pd.to_datetime(train1['Dates'])
train1['Date'] = Dt.dt.date
train1['Year'] = Dt.dt.year
train1['Month'] = Dt.dt.month
train1['Day'] = Dt.dt.day
train1['Hour'] = Dt.dt.hour
train1.drop(["Dates"], axis = 1, inplace = True)

DOW_mapping = {'Monday':1, 'Tuesday':1, 'Wednesday':1, 'Thursday':1, 'Friday':1, 'Saturday':0, 'Sunday':0}
train1['DayOfWeek'] = train1['DayOfWeek'].map(DOW_mapping)

count_encoding = train1['PdDistrict'].value_counts().to_dict()
train1['PdDistrict'] = train1['PdDistrict'].map(count_encoding)

Ad_dict = {'.* / .*':0, '.*':1}
train1['Address'] = train1['Address'].replace(Ad_dict,regex = True)

le = LabelEncoder()
train1['Category'] = le.fit_transform(train1['Category'])
train1
# DayOfWeek...平日休日mapping, PdDistrict...count

,Category,DayOfWeek,PdDistrict,Address,X,Y,Date,Year,Month,Day,Hour
0,37,1,105284,0,-122.425892,37.774599,2015-05-13,2015,5,13,23
1,21,1,105284,0,-122.425892,37.774599,2015-05-13,2015,5,13,23
2,21,1,105284,0,-122.424363,37.800414,2015-05-13,2015,5,13,23
3,16,1,105284,1,-122.426995,37.800873,2015-05-13,2015,5,13,23
4,16,1,49311,1,-122.438738,37.771541,2015-05-13,2015,5,13,23
...,...,...,...,...,...,...,...,...,...,...,...
878044,25,1,65592,0,-122.459033,37.714056,2003-01-06,2003,1,6,0
878045,16,1,78841,1,-122.447364,37.731948,2003-01-06,2003,1,6,0
878046,16,1,157174,0,-122.403390,37.780266,2003-01-06,2003,1,6,0
878047,35,1,157174,0,-122.390531,37.780607,2003-01-06,2003,1,6,0


In [6]:
Dt = pd.to_datetime(test['Dates'])
test['Date'] = Dt.dt.date
test['Year'] = Dt.dt.year
test['Month'] = Dt.dt.month
test['Day'] = Dt.dt.day
test['Hour'] = Dt.dt.hour
test.drop(["Dates"], axis = 1, inplace = True)

DOW_mapping = {'Monday':1, 'Tuesday':1, 'Wednesday':1, 'Thursday':1, 'Friday':1, 'Saturday':0, 'Sunday':0}
test['DayOfWeek'] = test['DayOfWeek'].map(DOW_mapping)


count_encoding = test['PdDistrict'].value_counts().to_dict()
test['PdDistrict'] = test['PdDistrict'].map(count_encoding)

Ad_dict = {'.* / .*':0, '.*':1}
test['Address'] = test['Address'].replace(Ad_dict,regex = True)

In [7]:
cols = ['DayOfWeek', 'PdDistrict', 'Address', 'X', 'Y', 'Year', 'Month', 'Day', 'Hour']

X = train1[cols]
Y = train1['Category']

X_test = test[cols]

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size = 0.3, random_state = 1219)

In [9]:
from sklearn.ensemble import RandomForestClassifier as RFC

rf = RFC()

rf.fit(X_train, y_train)

RandomForestClassifier()

In [10]:
y_pred = rf.predict(X_val)

In [11]:
from sklearn.metrics import accuracy_score
accuracy_score(y_val, y_pred)

0.2985364186867632

In [12]:
rf_submission = rf.predict(X_test)

In [14]:
import csv
pred = rf.predict_proba(test[['DayOfWeek', 'PdDistrict', 'Address', 'X', 'Y', 'Year', 'Month', 'Day', 'Hour']])
submission = pd.DataFrame(columns=['Id'], data=test)
submission = pd.concat([submission, pd.DataFrame(pred, columns=["ARSON", "ASSAULT", "BAD CHECKS", "BRIBERY", "BURGLARY", "DISORDERLY CONDUCT",
"DRIVING UNDER THE INFLUENCE", "DRUG/NARCOTIC", "DRUNKENNESS", "EMBEZZLEMENT", "EXTORTION",
"FAMILY OFFENSES", "FORGERY/COUNTERFEITING", "FRAUD", "GAMBLING", "KIDNAPPING", "LARCENY/THEFT",
"LIQUOR LAWS", "LOITERING", "MISSING PERSON", "NON-CRIMINAL", "OTHER OFFENSES", "PORNOGRAPHY/OBSCENE MAT",
"PROSTITUTION", "RECOVERED VEHICLE", "ROBBERY", "RUNAWAY", "SECONDARY CODES", "SEX OFFENSES FORCIBLE",
"SEX OFFENSES NON FORCIBLE", "STOLEN PROPERTY", "SUICIDE", "SUSPICIOUS OCC", "TREA", "TRESPASS",
"VANDALISM", "VEHICLE THEFT", "WARRANTS", "WEAPON LAWS"])], axis=1)

In [ ]:
submission.to_csv("rf3.csv", index=False)